In [1]:
import pandas as pd
import requests
import re

In [2]:
url = "https://en.wikipedia.org/wiki/2011_Irish_general_election"

In [3]:
data = requests.get(url)

In [6]:
tables = pd.read_html(data.content, flavor='bs4', match = "Polling Agency")

In [7]:
polling_data_2011 = tables[0]

In [8]:
polling_data_2011[0:20]

,Date,Source,Polling Agency,Fianna Fáil,Fine Gael,Labour Party,Green Party,Sinn Féin,Ind./Others
0,25 February 2011,General election,NaN,17.5%,36.1%,19.5%,1.8%,9.9%,15.4%
1,25 February 2011,RTÉ exit poll[111],Millward Brown Lansdowne,15.1%,36.1%,20.5%,2.7%,10.1%,15.5%
2,23 February 2011,Paddy Power[112],RED C,16%,40%,18%,3%,10%,14%
3,23 February 2011,Irish Independent[113],Millward Brown Lansdowne,14%,38%,20%,1%,11%,16%
4,21 February 2011,The Irish Times[114],Ipsos MRBI,16%,37%,19%,2%,11%,15%
5,20 February 2011,The Sunday Business Post[115],RED C,16%,39%,17%,2%,12%,14%
6,20 February 2011,Sunday Independent[115],Millward Brown Lansdowne,16%,37%,20%,1%,12%,14%
7,17 February 2011,Irish Daily Star[116],OI Research,17%,39%,18%,2%,10%,14%
8,16 February 2011,Irish Independent[117],Millward Brown Lansdowne,12%,38%,23%,1%,10%,16%
9,13 February 2011,The Sunday Business Post[118],RED C,15%,38%,20%,3%,10%,14%


In [9]:
day_re = "[0-9]+"
month_re = "[A-Z][a-z]+"
year_re = "[0-9]{4}"
for index, value in polling_data_2011["Date"].items():
    #day
    match = re.search(day_re, value)
    if match:
        polling_data_2011.loc[index,"Poll_day"] = match.group()
      
    #month
    match = re.search(month_re, value)
    if match:
        polling_data_2011.loc[index, "Poll_month"] = match.group()
    
    #year
    match = re.search(year_re, value)
    if match:
        polling_data_2011.loc[index, "Poll_year"] = match.group()

In [10]:
regex_for_reference = "\["
for index, value in polling_data_2011["Source"].items():
    ref_match = re.search(regex_for_reference, value)
    if ref_match:
        location = ref_match.span()[0]
        new_name = (value[0:location])
        polling_data_2011.loc[index,"Source"] = new_name
    else:
        polling_data_2011.loc[index,"Source"] = value

In [11]:
polling_data_2011["Source"].unique()

array(['General election', 'RTÉ exit poll', 'Paddy Power',
       'Irish Independent', 'The Irish Times', 'The Sunday Business Post',
       'Sunday Independent', 'Irish Daily Star',
       'The Sunday Business Post.', 'The Irish Sun', 'TV3 News',
       'European election', 'Local elections'], dtype=object)

In [12]:
regex_for_period = "\."
for index, value in polling_data_2011["Source"].items():
    period_match = re.search(regex_for_period, value)
    
    if period_match:
        location = period_match.span()[0]
        new_name = (value[0:location])
        polling_data_2011.loc[index,"Source"] = new_name
    else:
        polling_data_2011.loc[index,"Source"] = value

In [13]:
regex_to_find_percent = "%"
columns = ["Fianna Fáil", "Fine Gael", "Labour Party", "Green Party", "Sinn Féin", "Ind./Others"]
for column in columns:
    for index, value in polling_data_2011[column].items():
        match = re.search(regex_to_find_percent, value)
        if match:
            match_index = match.span()[0]
            new_value = (value[0:match_index])
            polling_data_2011.loc[index, column] = new_value

In [14]:
for column in columns:
    polling_data_2011[column] = polling_data_2011[column].astype(float)

In [15]:
polling_data_2011.describe()

,Fianna Fáil,Fine Gael,Labour Party,Green Party,Sinn Féin,Ind./Others
count,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000
mean,24.473239,32.447887,18.826761,4.132394,9.359155,10.746479
std,8.397091,3.762441,5.838394,2.200894,2.228618,2.964785
min,12.000000,23.000000,9.000000,1.000000,4.000000,6.000000
25%,17.000000,30.000000,14.850000,2.000000,8.000000,9.000000
50%,24.000000,33.000000,19.000000,4.000000,9.000000,10.000000
75%,28.000000,35.000000,23.000000,5.500000,10.000000,12.500000
max,42.000000,40.000000,35.000000,9.000000,16.000000,19.700000


In [16]:
election_regex = "[Ee]lection"
exit_poll_regex = "[Ee]xit poll"
for index, value in polling_data_2011["Source"].items():
    election_match = re.search(election_regex, value)
    exit_poll_match = re.search(exit_poll_regex, value)
    if election_match: 
        polling_data_2011.loc[index, "Poll_type"] = "E"
    if exit_poll_match:
        polling_data_2011.loc[index, "Poll_type"] = "EP"
    if not election_match and not exit_poll_match:
        polling_data_2011.loc[index, "Poll_type"] = "OP"

In [17]:
polling_data_2011.pivot_table(index=["Poll_year","Poll_type"])

Fianna Fáil  Fine Gael  Green Party  Ind./Others  \
Poll_year Poll_type                                                     
2007      E            41.500000  27.300000     4.700000     9.400000   
          OP           36.000000  29.000000     7.000000     9.000000   
2008      OP           35.785714  29.285714     6.500000     8.428571   
2009      E            24.750000  30.650000     2.100000    18.850000   
          OP           23.000000  33.437500     4.750000    10.187500   
2010      OP           21.588235  31.705882     3.352941     9.235294   
2011      E            17.500000  36.100000     1.800000    15.400000   
          EP           15.100000  36.100000     2.700000    15.500000   
          OP           15.600000  36.200000     1.866667    14.066667   

                     Labour Party  Sinn Féin  
Poll_year Poll_type                           
2007      E             10.100000   6.900000  
          OP            12.250000   7.000000  
2008      OP            11.714286   8.285714  
2009      E             14.300000   9.300000  
          OP            19.937500   8.687500  
2010      OP            24.470588   9.588235  
2011      E             19.500000   9.900000  
          EP            20.500000  10.100000  
          OP            20.666667  11.533333